In [21]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, show
import numpy as np
from scipy.interpolate import interp1d
from bokeh.models import HoverTool, NumeralTickFormatter, Range1d, Label
from bokeh.models.widgets import RadioButtonGroup
from bokeh.layouts import widgetbox

In [2]:
df = pd.read_csv("subway_inequality.csv")

In [44]:
output_notebook()

COLORS = {
    ('1', '2', '3'): '#FF3535',
    ('4', '5', '6'): '#019733',
    ('7'): '#CC02C8',
    ('A', 'C', 'E'): '#0F6797',
    ('B', 'D', 'F', 'M'): '#FF9800',
    ('N', 'Q', 'R'): '#ffe400',
    ('J', 'Z'): '#986701',
    ('L'): '#999999',
    ('G'): '#9BCF00',
}

def find_color(line):
    for lines, color in COLORS.items():
        if line in lines:
            return color

def draw_line(line='1'):
    line_data = df[df.line == line]
    income2011 = line_data.income2011

    source = dict(
                position=line_data.position,
                income=income2011,
                subway_stop=line_data.stop_name,
            )

    hover = HoverTool(
            tooltips=[
                ("Income", "$@income"),
                ("Subway Stop", "@subway_stop"),
            ]
        )

    fig = figure(tools = [hover])

    smoothed_x = np.arange(1,max(line_data.position),0.05)
    smoothed_fy = interp1d(line_data.position,income2011, kind = 'cubic')

    color = find_color(line)
    fig.line(x=smoothed_x, y=smoothed_fy(smoothed_x),line_width=3, color=color)
    fig.circle('position','income',size=8,fill_color="black",source=source, line_color=color)
    fig.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
    fig.y_range = Range1d(0, 250000)
    fig.x_range = Range1d(0, max(line_data.position) + 5)
    
    prev_county_name = None
    for position, county_name in zip(line_data.position, line_data.county_name):
        if county_name != prev_county_name:
            label = Label(x=position, y=10000, text=county_name)
            fig.add_layout(label)
        prev_county_name = county_name

    show(fig)

Loading BokehJS ...

In [45]:
from ipywidgets import interact

interact(draw_line, line=[line for lines in COLORS for line in lines])

<function __main__.draw_line>